In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile /content/drive/MyDrive/flask_app/app.py
from flask import Flask, request, render_template, url_for
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

app = Flask(__name__)

# Folder to save uploaded images
UPLOAD_FOLDER = '/content/drive/MyDrive/flask_app/static'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

model = load_model('/content/drive/MyDrive/flask_app/land_classification_model.h5')

classes = ["agriculture", "water", "urban", "desert", "roads", "trees"]

def prepare_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (64,64))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = ''
    image_url = ''

    if request.method == 'POST':
        file = request.files['file']
        if file:
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(file_path)

            # Prepare image for model
            img = prepare_image(file_path)
            pred = model.predict(img)
            pred_class = classes[np.argmax(pred)]
            prediction = f"Predicted Class: {pred_class}"

            # Flask static URL
            image_url = url_for('static', filename=file.filename)

    return render_template('index.html', prediction=prediction, image=image_url)

if __name__ == '__main__':
    app.run()


Overwriting /content/drive/MyDrive/flask_app/app.py


In [ ]:
%%writefile /content/drive/MyDrive/flask_app/templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Land Classification App</title>
    <style>
        body {
            margin: 0;
            padding: 0;
            font-family: Arial, sans-serif;
            background: linear-gradient(135deg, #2c5364, #203a43, #0f2027);
            color: white;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            text-align: center;
        }
        .container {
            background: rgba(255, 255, 255, 0.1);
            padding: 40px;
            border-radius: 20px;
            width: 450px;
            box-shadow: 0 0 20px rgba(0,0,0,0.3);
            backdrop-filter: blur(5px);
        }
        h1 {
            margin-bottom: 20px;
            font-size: 28px;
            letter-spacing: 1px;
        }
        input[type="file"] {
            margin: 20px 0;
            padding: 10px;
            background: #fff;
            border-radius: 10px;
            cursor: pointer;
        }
        button {
            background: linear-gradient(45deg, #00c6ff, #0072ff);
            border: none;
            padding: 12px 25px;
            font-size: 16px;
            color: white;
            border-radius: 30px;
            cursor: pointer;
            transition: 0.3s;
        }
        button:hover {
            background: linear-gradient(45deg, #0072ff, #00c6ff);
            transform: scale(1.05);
        }
        img {
            margin-top: 20px;
            border-radius: 10px;
            border: 3px solid #fff;
        }
        .prediction-box {
            margin-top: 20px;
            padding: 15px;
            background: rgba(0,0,0,0.4);
            border-radius: 10px;
            font-size: 18px;
            font-weight: bold;
        }
    </style>
</head>
<body>

    <div class="container">
        <h1>Land Classification App</h1>

        <form method="post" enctype="multipart/form-data">
            <input type="file" name="file" required>
            <br>
            <button type="submit">Predict</button>
        </form>

        {% if prediction %}
            <div class="prediction-box">
                Prediction: {{ prediction }}
            </div>
        {% endif %}

        {% if image %}
            <img src="{{ image }}" width="300">
        {% endif %}
    </div>

</body>
</html>


Overwriting /content/drive/MyDrive/flask_app/templates/index.html


In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2wRfT0alzNfJx6qLXV2vP3dg91A_2sMwmDqQQgiioDxtgPWc5")


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print(" * NGROK URL:", public_url)

!python /content/drive/MyDrive/flask_app/app.py


 * NGROK URL: NgrokTunnel: "https://ba9de64d791f.ngrok-free.app" -> "http://localhost:5000"
2025-11-30 12:14:18.610643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764504858.648845    3651 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764504858.659937    3651 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764504858.690861    3651 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764504858.690918    3651 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than o